In [13]:
# Блок 1: Импорт библиотек
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import StandardScaler


In [14]:
# Блок 2: Загрузка данных
data = pd.read_excel('yvc.xlsx')
df = pd.DataFrame(data)

print(df)

    ув  ус    х1    х2    х3         х4    х5       х6   х7    х8    х9  \
0   26  27  9.20  11.1  4.30  91.631206   2.0   9.7000  256  34.0  0.90   
1   28  27  9.40  10.5  4.15  88.652482   8.0   9.9000  256  31.0  0.50   
2   25  27  9.10  10.9  4.10  91.835206   2.0   9.6000  256  31.0  0.57   
3   26  24  9.50  11.2  4.25  90.229885   2.0  10.0000  294  31.0  0.70   
4   28  29  9.00  10.8  4.40  92.307692   2.0   9.5000  313  33.0  0.66   
..  ..  ..   ...   ...   ...        ...   ...      ...  ...   ...   ...   
85  29  28  9.80  10.5  4.25  91.050725   2.0  10.3300  341  28.0  0.95   
86  26  29  9.95  10.0  3.90  92.152778   2.0   9.9500  256  32.0  0.60   
87  23  26  9.40  11.2  4.25  92.912281   3.0   9.9000  277  30.0  0.70   
88  26  30  9.70  12.0  4.40  90.694444   0.1  10.2378  210  33.2  0.82   
89  25  26  8.90  11.9  4.40  92.061856  16.0   9.4000  349  29.0  0.60   

          х10  
0   51.232390  
1   50.831217  
2   50.831217  
3   50.230144  
4   50.229696  
.. 

In [15]:
# Блок 3: Определение входных и выходных данных
headers = df.columns.tolist()
print(headers)

X = df[['х1', 'х2', 'х3', 'х4', 'х5', 'х6', 'х7', 'х8', 'х9', 'х10']]
y = df['ув']

['ув', 'ус', 'х1', 'х2', 'х3', 'х4', 'х5', 'х6', 'х7', 'х8', 'х9', 'х10']


In [16]:
# Блок 4: Разделение данных на обучающую, тестовую и валидационную выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=737)

In [17]:
# Нормализация данных: приведение всех признаков к одному масштабу
scaler = StandardScaler()  # Создание объекта для нормализации
X_train_scaled = scaler.fit_transform(X_train)  # Обучение и преобразование обучающего набора данных
X_test_scaled = scaler.transform(X_test)  # Преобразование тестового набора данных

In [18]:
# Блок 5: Сетка гиперпараметров и кросс-валидация
# Задание сетки гиперпараметров для модели MLPRegressor
# Уточненная сетка параметров
from sklearn.model_selection import ShuffleSplit

param_grid = {
    'activation': ['tanh'],
    'alpha': [0.8, 0.9, 1.0],
    'epsilon': [1e-10],
    'hidden_layer_sizes': [(10,), (15,), (20,)],
    'learning_rate': ['constant'],
    'learning_rate_init': [0.0001, 0.0005, 0.001],
    'momentum': [0.5, 0.75, 1.00],
    'n_iter_no_change': [15],
    'nesterovs_momentum': [True],
    'solver': ['sgd'],
    'tol': [0.0001, 0.001]
}

# Настройка кросс-валидации с 5 фолдами
cv = ShuffleSplit(
    n_splits=5,         # количество фолдов
    test_size=0.25,       # перемешивание данных
    random_state=37     # для воспроизводимости результатов
)

# GridSearchCV для подбора гиперпараметров модели
grid_search = GridSearchCV(
    estimator=MLPRegressor(random_state=24),
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=cv,
    verbose=2,
    n_jobs=-1
)


In [19]:
# Блок 6: Обучение GridSearchCV
print("Начинается подбор гиперпараметров...")
grid_search.fit(X_train_scaled, y_train)  # Обучение модели с подбором гиперпараметров

Начинается подбор гиперпараметров...
Fitting 5 folds for each of 162 candidates, totalling 810 fits


c:\Users\Серж\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


GridSearchCV(cv=ShuffleSplit(n_splits=5, random_state=37, test_size=0.25, train_size=None),
             estimator=MLPRegressor(random_state=24), n_jobs=-1,
             param_grid={'activation': ['tanh'], 'alpha': [0.8, 0.9, 1.0],
                         'epsilon': [1e-10],
                         'hidden_layer_sizes': [(10,), (15,), (20,)],
                         'learning_rate': ['constant'],
                         'learning_rate_init': [0.0001, 0.0005, 0.001],
                         'momentum': [0.5, 0.75, 1.0], 'n_iter_no_change': [15],
                         'nesterovs_momentum': [True], 'solver': ['sgd'],
                         'tol': [0.0001, 0.001]},
             scoring='neg_mean_squared_error', verbose=2)

In [20]:
# Блок 7: Лучшая модель
best_params = grid_search.best_params_  # Получение лучших гиперпараметров

# Получение модели с лучшими параметрами
best_model = grid_search.best_estimator_

# Прогнозирование на тестовом наборе данных с лучшей моделью
y_pred_best = best_model.predict(X_test_scaled)

In [21]:
# Блок 8: Вычисление метрик
# Вычисление MAPE (Mean Absolute Percentage Error) для оценки точности
mape_best = mean_absolute_percentage_error(y_test, y_pred_best) * 100
# Вычисление RMSE (Root Mean Squared Error) для оценки отклонений предсказаний от реальных значений
rmse_best = np.sqrt(mean_squared_error(y_test, y_pred_best))

print("Лучшие параметры:", grid_search.best_params_)
print("Обучающая выборка:")
print(f"MAPE: {-grid_search.best_score_} %")
# Вывод лучших метрик
print("Тестовая выборка:")
print(f"MAPE: {mape_best:.2f}%")
print(f"RMSE: {rmse_best:.2f}")

Лучшие параметры: {'activation': 'tanh', 'alpha': 1.0, 'epsilon': 1e-10, 'hidden_layer_sizes': (15,), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'momentum': 0.75, 'n_iter_no_change': 15, 'nesterovs_momentum': True, 'solver': 'sgd', 'tol': 0.0001}
Обучающая выборка:
MAPE: 9.660915298005934 %
Тестовая выборка:
MAPE: 9.42%
RMSE: 2.73


In [22]:
# Блок 9: Создание DataFrame для тестовых значений
# Сравнение истинных значений с предсказанными для тестового набора
test_comparison_model = pd.DataFrame({"yв": y_test.values, "y_pred_best": y_pred_best})
print(test_comparison_model)

    yв  y_pred_best
0   28    23.770241
1   19    24.891259
2   26    25.440863
3   23    23.088839
4   25    23.838286
5   23    24.784727
6   26    24.099520
7   27    24.764106
8   25    24.545947
9   28    24.326188
10  23    25.669259
11  25    22.732983
12  22    24.028050
13  22    24.775773
14  29    24.917958
15  24    24.661307
16  26    23.770479
17  28    25.166971


In [23]:
# Блок 10: Сохранение результатов
# Сохранение результатов и метрик в Excel файл
test_comparison_best = pd.DataFrame({"ув": y_test.values, "y_pred_py": y_pred_best})
metrics_best = pd.DataFrame({"Метрика": ["MAPE", "RMSE"], "Значение": [mape_best, rmse_best]})

# Запись в Excel
output_file = 'models_results_optimized_ув.xlsx'  # Имя выходного файла
with pd.ExcelWriter(output_file) as writer:
    test_comparison_best.to_excel(writer, sheet_name='Результаты лучшей модели', index=False)  # Результаты модели
    metrics_best.to_excel(writer, sheet_name='Метрики лучшей модели', index=False)  # Метрики модели